In [4]:
#simple genai app with open ai

import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [5]:
#data ingestion
#scraping the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/prompt_engineering/how_to_guides/playground/managing_model_configurations")

In [7]:
loader

In [8]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/playground/managing_model_configurations', 'title': 'Managing Model Configurations | 🦜️🛠️ LangSmith', 'description': 'The LangSmith playground allows you to save and manage your model configurations, making it easy to reuse your preferred settings across sessions.', 'language': 'en'}, page_content="\n\n\n\n\nManaging Model Configurations | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringTutorialsOptimize a classifierHow-to GuidesPlaygroundRun the playground against a custom LangServe model serverRun the playground against an OpenAI-compliant model provider/proxyUse custom TLS certificatesManaging Model ConfigurationsTesting over a datasetPromptsConceptual GuideDeployment (LangGraph Platform)A

In [9]:
#divide these documents into chunks
#convert these chunks to vectors
#do vector embedding
#store in vector store db

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)


In [10]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/playground/managing_model_configurations', 'title': 'Managing Model Configurations | 🦜️🛠️ LangSmith', 'description': 'The LangSmith playground allows you to save and manage your model configurations, making it easy to reuse your preferred settings across sessions.', 'language': 'en'}, page_content='Managing Model Configurations | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/playground/managing_model_configurations', 'title': 'Managing Model Configurations | 🦜️🛠️ LangSmith', 'description': 'The LangSmith playground allows you to save and manage your model configurations, making it easy to reuse your preferred settings across sessions.', 'language': 'en'}, page_content='Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegio

In [11]:
#convert these chunks to vectors
#cosine simi.arity is used

from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings() 

In [12]:
#storing in FAISS db

from langchain_community.vectorstores import FAISS

vectorstoredb=FAISS.from_documents(documents,embeddings)



In [13]:
vectorstoredb

In [14]:
#now based on similarity search we wll query anything from the db and provide with an appropriate response

query= "Log your first trace"
result=vectorstoredb.similarity_search(query)

result[0].page_content

'Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringTutorialsOptimize a classifierHow-to GuidesPlaygroundRun the playground against a custom LangServe model serverRun the playground against an OpenAI-compliant model provider/proxyUse custom TLS certificatesManaging Model ConfigurationsTesting over a datasetPromptsConceptual GuideDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referencePrompt EngineeringHow-to GuidesPlaygroundManaging Model ConfigurationsOn this pageManaging Model Configurations\nThe LangSmith playground allows you to save and manage your model configurations, making it easy to reuse your preferred settings across sessions.\nSaving a Configuration\u

In [16]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
#got error for ChatOpenAI(model="gpt-4o",api_key="OPENAI_API_KEY")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x00000256FA0E0F70> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000256FA0E3A60> root_client=<openai.OpenAI object at 0x00000256A642C6D0> root_async_client=<openai.AsyncOpenAI object at 0x00000256FA0E0EE0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [17]:
#retreival chain
 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
answer the following question based only on the provided context
<context>
{context}
</context




"""
)

#document chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain



RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nanswer the following question based only on the provided context\n<context>\n{context}\n</context\n\n\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000256FA0E0F70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000256FA0E3A60>, root_client=<openai.OpenAI object at 0x00000256A642C6D0>, root_async_client=<openai.AsyncOpenAI object at 0x00000256FA0E0EE0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_

In [18]:
from langchain_core.documents import Document 
document_chain.invoke({
    "input":"number of ways to login the langchain",
    "context":[Document(page_content="how to view the traces")]
})

"To view the traces based on the provided context, you would need to follow the specific instructions or use the tools mentioned within that context. However, since the context does not provide detailed steps or specify particular tools or methods, I can't give a precise answer. Generally, viewing traces could involve using software or tools designed for trace analysis, such as logging frameworks, monitoring dashboards, or development environments that support trace visualization. If more context or details were provided, I could offer a more specific answer."

In [19]:
#retreiver
# we want documents to first con=me from retreiver we just set up. that way we can use the retreiver to dynamically select the most relevant documents and pass those in for a given question

retreiver=vectorstoredb.as_retriever()

In [20]:
from langchain.chains import create_retrieval_chain
retreival_chain= create_retrieval_chain(retreiver,document_chain)


In [21]:
retreival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000256A642C640>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nanswer the following question based only on the provided context\n<context>\n{context}\n</context\n\n\n\n\n'), additional_kwargs={})])
            

In [24]:
response=retreival_chain.invoke(
    {"input":"number of ways to login the langchain"}
    
)

In [25]:
response['answer']

'To delete a configuration, you need to follow these steps:\n\n1. Select the configuration you want to remove.\n2. Click the trash can icon to delete it.'

In [26]:
response

{'input': 'number of ways to login the langchain',
 'context': [Document(id='fa639433-0cb4-4eda-8268-e17a49b86c75', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/playground/managing_model_configurations', 'title': 'Managing Model Configurations | 🦜️🛠️ LangSmith', 'description': 'The LangSmith playground allows you to save and manage your model configurations, making it easy to reuse your preferred settings across sessions.', 'language': 'en'}, page_content='Deleting Configurations\u200b\n\nSelect the configuration you want to remove\nClick the trash can icon to delete it\n\nResetting to Default\u200b\n\nTo restore default model settings: Click the x button in the top row\nWas this page helpful?You can leave detailed feedback on GitHub.PreviousUse custom TLS certificatesNextTesting over a datasetSaving a ConfigurationManaging Saved ConfigurationsAccessing Saved ConfigurationsEditing ConfigurationsDeleting ConfigurationsResetting to DefaultCommuni

In [27]:
response['context']

[Document(id='fa639433-0cb4-4eda-8268-e17a49b86c75', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/how_to_guides/playground/managing_model_configurations', 'title': 'Managing Model Configurations | 🦜️🛠️ LangSmith', 'description': 'The LangSmith playground allows you to save and manage your model configurations, making it easy to reuse your preferred settings across sessions.', 'language': 'en'}, page_content='Deleting Configurations\u200b\n\nSelect the configuration you want to remove\nClick the trash can icon to delete it\n\nResetting to Default\u200b\n\nTo restore default model settings: Click the x button in the top row\nWas this page helpful?You can leave detailed feedback on GitHub.PreviousUse custom TLS certificatesNextTesting over a datasetSaving a ConfigurationManaging Saved ConfigurationsAccessing Saved ConfigurationsEditing ConfigurationsDeleting ConfigurationsResetting to DefaultCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHome

In [29]:
## similarly using ollama

#in git hub all models info is there


In [35]:
import os
from dotenv import load_dotenv
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama


load_dotenv()

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

#prompt template

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are an Professor. Provide me answers based on questions"),
        ("user","Question:{question}")
    ]
)

#streamlit framework

#st.title("Langchain demo with OLLAMA")
#input_text=st.text_input("What is your question?")

##ollama Llama2 model
llm=Ollama(model="gemma:2b")
output_parser=StrOutputParser()
chain=prompt|llm|output_parser

#now if i put any text in the input text the below will get executed and run the llm model
#if input_text:
 #   st.write(chain.invoke({"Question":input_text}))

#streamlit run app.py
#make sure you are in correct folder location before running the model